In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os 
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [2]:
load_dotenv()

True

In [3]:
key = os.getenv("OPENAI_API_KEY")

In [4]:
llm = ChatOpenAI(openai_api_key = key, model= 'gpt-3.5-turbo', temperature=0.7)

c:\Users\Gaurav\OneDrive\Desktop\Projects\LLM Projects\MCQ Generator\MCQ-Generator-Using-LLM\env\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002285A176510>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002285DC2A4D0>, openai_api_key='sk-None-rxpex7H2zzgPTlgeht9sT3BlbkFJ9MFJAanKgWEw9W8WEh5f', openai_proxy='')

In [6]:
with open(r"C:\Users\Gaurav\OneDrive\Desktop\Projects\LLM Projects\MCQ Generator\MCQ-Generator-Using-LLM\response.json") as f:
    RESPONSE_JSON = json.load(f)

In [7]:
print(RESPONSE_JSON)

{'1': {'No': '1', 'MCQ': 'Multiple choice questions', 'Options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'Correct': 'Correct Answer'}, '2': {'No': '1', 'MCQ': 'Multiple choice questions', 'Options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'Correct': 'Correct Answer'}, '3': {'No': '1', 'MCQ': 'Multiple choice questions', 'Options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'Correct': 'Correct Answer'}, '4': {'No': '1', 'MCQ': 'Multiple choice questions', 'Options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'Correct': 'Correct Answer'}}


In [8]:
TEMPLATE = """
Text : {text}
You are a expert in MCQ maker. Given the above text, it is your job to create quiz \
of {number} multiple choice question for subject in {tone}. Make sure the questions are not repeated and check all the questions to be conforming.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
ensure to make {number} MCQ

### RESPONSE_JSON
{RESPONSE_JSON}
"""

In [9]:
quiz_generation_prompt=PromptTemplate(
    input_variables = ["text", "numbers", "tone", "response_json"],
    template=TEMPLATE
)

In [10]:
from langchain.chains import LLMChain

# Assuming `llm` and `quiz_generation_prompt` are already defined
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


c:\Users\Gaurav\OneDrive\Desktop\Projects\LLM Projects\MCQ Generator\MCQ-Generator-Using-LLM\env\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [11]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""


In [12]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)


In [13]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [14]:

generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
                                        output_variables=["quiz", "review"], verbose=True,)


In [15]:
PATH = r"C:\Users\Gaurav\OneDrive\Desktop\Projects\LLM Projects\MCQ Generator\MCQ-Generator-Using-LLM\data.txt"

with open(PATH, 'r', encoding='utf-8') as file:
    TEXT = file.read()


In [16]:
TEXT

'Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is artificial intelligence capable of generating text, images, videos, or other data using generative models,[2] often in response to prompts.[3][4] Generative AI models learn the patterns and structure of their input training data and then generate new data that has similar characteristics.[5][6]\n\nImprovements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI generators such as Sora.[7][8][9][10] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed generative AI models.[3][11][12]\n\nGenerative AI has uses across a wide range of industries, including software

In [17]:
NUMBER=5 
SUBJECT="AI"
TONE="simple"
    

In [18]:
# Calling Chains

with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "RESPONSE_JSON": json.dumps(RESPONSE_JSON)
        }
        )

c:\Users\Gaurav\OneDrive\Desktop\Projects\LLM Projects\MCQ Generator\MCQ-Generator-Using-LLM\env\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text : Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is artificial intelligence capable of generating text, images, videos, or other data using generative models,[2] often in response to prompts.[3][4] Generative AI models learn the patterns and structure of their input training data and then generate new data that has similar characteristics.[5][6]

Improvements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI generators such as Sora.[7][8][9][10] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed gener

In [19]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2613
Prompt Tokens:2139
Completion Tokens:474
Total Cost:0.0041565000000000005


In [20]:
response

{'text': 'Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is artificial intelligence capable of generating text, images, videos, or other data using generative models,[2] often in response to prompts.[3][4] Generative AI models learn the patterns and structure of their input training data and then generate new data that has similar characteristics.[5][6]\n\nImprovements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI generators such as Sora.[7][8][9][10] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed generative AI models.[3][11][12]\n\nGenerative AI has uses across a wide range of industries, including

In [21]:
quiz=response.get("quiz")

In [22]:
print(quiz)


### RESPONSE_JSON
{
    "1": {
        "No": "1",
        "MCQ": "When was the academic discipline of artificial intelligence established?",
        "Options": {
            "a": "1950",
            "b": "1956",
            "c": "1970",
            "d": "1980"
        },
        "Correct": "b: 1956"
    },
    "2": {
        "No": "2",
        "MCQ": "Which network enabled advancements in generative models in 2017?",
        "Options": {
            "a": "Markov Chain",
            "b": "Variational Autoencoder",
            "c": "Transformer",
            "d": "Generative Adversarial Network"
        },
        "Correct": "c: Transformer"
    },
    "3": {
        "No": "3",
        "MCQ": "Which company released GPT-4 in March 2023?",
        "Options": {
            "a": "OpenAI",
            "b": "Microsoft",
            "c": "Google",
            "d": "Meta"
        },
        "Correct": "b: Microsoft"
    },
    "4": {
        "No": "4",
        "MCQ": "Which country is leading 

In [23]:
import json

quiz = json.loads(quiz)

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["MCQ"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["Options"].items()
            ]
        )
    correct = value["Correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

[{'MCQ': 'What is generative artificial intelligence capable of generating?',
  'Choices': 'a: Text, images, videos, or other data | b: Only text | c: Only images | d: Only videos',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is not an example of a generative AI system mentioned in the text?',
  'Choices': 'a: ChatGPT | b: Copilot | c: Google Translate | d: DALL-E',
  'Correct': 'c'},
 {'MCQ': 'What concerns have been raised about the potential misuse of generative AI?',
  'Choices': 'a: Cybercrime | b: Mass replacement of human jobs | c: Use of fake news or deepfakes | d: All of the above',
  'Correct': 'd'},
 {'MCQ': 'Which year saw the release of GPT-4, which some argue could be viewed as an early version of an artificial general intelligence (AGI) system?',
  'Choices': 'a: 2020 | b: 2021 | c: 2022 | d: 2023',
  'Correct': 'd'},
 {'MCQ': 'According to a survey, which country is leading the world in adopting generative AI?',
  'Choices': 'a: United States | b: India | c: Chi

In [ ]:
quiz=pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("GenerativAI.csv",index=False)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'07_16_2024_15_32_22'

In [ ]:
LOG_FILE=f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"


In [ ]:
LOG_FILE

'07_16_2024_15_38_33.log'

In [ ]:
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'07_16_2024_15_39_57'